# Online Predictions with TensorFlow Serving

## Import project dependencies

In [1]:
import os
import json
import requests
import subprocess
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

## Loading the saved model to local filesystem

In [2]:
!gsutil cp -r gs://tensorflow-ml-course-blob/models .

Copying gs://tensorflow-ml-course-blob/models/1/saved_model.pb...
Copying gs://tensorflow-ml-course-blob/models/1/variables/variables.data-00000-of-00001...
Copying gs://tensorflow-ml-course-blob/models/1/variables/variables.index...    
- [3 files][ 43.1 KiB/ 43.1 KiB]                                                
Operation completed over 3 objects/43.1 KiB.                                     


## Defining the model directory

In [3]:
MODEL_DIR = "models/"

## Setting up the production environment

### Exporting the MODEL_DIR to environment variables

In [4]:
os.environ['MODEL_DIR'] = os.path.abspath(MODEL_DIR)

### Running the TensorFlow Serving REST API

In [5]:
%%bash --bg
nohup tensorflow_model_server --rest_api_port=8507 --model_name=flights_logistic --model_base_path="${MODEL_DIR}" >server.log 2>&1


In [6]:
!tail server.log

2019-09-03 15:15:36.616906: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2019-09-03 15:15:36.616936: I external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-20190902-224346): /proc/driver/nvidia/version does not exist
2019-09-03 15:15:36.746355: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:202] Restoring SavedModel bundle.
2019-09-03 15:15:37.083602: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:151] Running initialization op on SavedModel bundle at path: /home/jupyter/models/1
2019-09-03 15:15:37.089279: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:311] SavedModel load for tags { serve }; Status: success. Took 507278 microseconds.
2019-09-03 15:15:37.089380: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:103] No warmup data file found at /home/jupyte

## Requesting Predictions over HTTP

In [7]:
%%bigquery sample_prediction
SELECT * FROM `tensorflow-ml-course.flights.new_flights` LIMIT 1

In [8]:
sample_prediction.to_numpy().tolist()

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]

### Creating the JSON data object

In [9]:
data = json.dumps({"instances":sample_prediction.to_numpy().tolist()})

In [10]:
data

'{"instances": [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]}'

### Sending the POST request

In [11]:
headers = {"content-type":"application/json"}

In [13]:
json_response = requests.post(url="http://localhost:8507/v1/models/flights_logistic:predict", data=data, headers=headers)



In [14]:
json_response

<Response [200]>

In [15]:
predictions = json.loads(json_response.text)['predictions']

In [16]:
predictions

[[0.0657766461]]

### Sending the POST request to a specific model version

In [18]:
json_response_version = requests.post(url="http://localhost:8507/v1/models/flights_logistic/versions/1:predict", data=data, headers=headers)

In [19]:
predictions = json.loads(json_response_version.text)['predictions']

In [20]:
predictions

[[0.0657766461]]